In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [12]:
ssem = pd.read_excel('./data/corr/삼성전기_total.xlsx')

In [13]:
ssem = ssem.dropna(axis = 1)

In [14]:
ssem.head()

,회사명,거래소코드,회계년도,[공통]총자본증가율(IFRS연결),[공통]자기자본증가율(IFRS연결),[공통]매출액증가율(IFRS연결),[공통]정상영업이익증가율(IFRS연결),[공통]순이익증가율(IFRS연결),[공통]총포괄이익증가율(IFRS연결),[공통]매출액정상영업이익률(IFRS연결),[공통]매출액순이익률(IFRS연결),[공통]총자본사업이익률(IFRS연결),[공통]총자본정상영업이익률(IFRS연결),[공통]총자본순이익률(IFRS연결),[공통]자기자본정상영업이익률(IFRS연결),[공통]자기자본순이익률(IFRS연결),[공통]자본금정상영업이익률(IFRS연결),[공통]자본금순이익률(IFRS연결),[공통]영업비용 대 영업수익비율(IFRS연결),[공통]수지비율(관계기업투자손익 제외)(IFRS연결),[공통]세금과공과 대 총비용비율(IFRS연결),[공통]유보율(IFRS연결),[공통]적립금비율(재정비율)(IFRS연결),[공통]1주당매출액(IFRS연결),[공통]1주당순이익(IFRS연결),[공통]1주당 CASH FLOW(IFRS연결),[공통]1주당정상영업이익(IFRS연결),[공통]자기자본구성비율(IFRS연결),[공통]타인자본구성비율(IFRS연결),[공통]부채비율(IFRS연결),[공통]차입금의존도(IFRS연결),[공통]차입금비율(IFRS연결),[공통]유보액대비율(IFRS연결),[공통]CASH FLOW 대 총자본비율(IFRS연결),[공통]총자본회전률(IFRS연결),[공통]자기자본회전률(IFRS연결),[공통]자본금회전률(IFRS연결),[공통]타인자본회전률(IFRS연결),[공통]부가가치(IFRS연결),[공통]총자본투자효율(IFRS연결),[공통]부가가치율(IFRS연결),[공통]노동소득분배율(IFRS연결),[공통]자본분배율(IFRS연결),거래년도(*),거래월(*),거래일수,최고가,최저가,산술평균,종가,거래량,거래대금
0,삼성전기(주),9150,2009/12,45.44,32.81,29.55,252.08,490.78,0.00,8.69,5.86,10.80,10.24,6.90,20.00,12.48,124.27,73.14,95.31,92.41,0,574.07,71.42,71526.62,3685,9537.68,6213.66,50.50,49.50,98.01,30.80,61.00,43.91,10.54,1.18,2.30,14.31,2.42,1557567.99,27.91,28.06,44.92,55.08,2009,12,21,108000,93000,100319,107500,17314291,17342602775
1,삼성전기(주),9150,2010/06,0.00,0.00,0.00,0.00,0.00,0.00,12.19,10.92,14.00,13.08,11.72,26.41,20.51,221.80,156.43,92.19,86.68,0,667.48,70.89,45491.18,3926,5254.64,5544.99,49.51,50.49,102.00,28.78,58.13,43.90,3.10,1.07,2.17,18.20,2.12,501580.64,7.62,14.21,0.00,0.00,2010,6,21,158500,133000,146571,154500,15957107,23273813814
2,삼성전기(주),9150,2010/12,22.48,33.90,25.55,61.37,95.45,58.00,11.17,9.57,13.11,12.53,10.74,23.50,18.27,200.54,142.95,92.65,88.71,0,798.81,66.83,89804.36,7171,12436.72,10027.15,55.63,44.37,79.76,25.01,44.95,50.13,12.47,1.12,2.10,17.96,2.40,2257211.70,33.02,32.39,38.85,61.15,2010,12,22,131500,121000,126114,124000,16677355,21039882720
3,삼성전기(주),9150,2011/06,8.55,18.08,-3.82,-58.35,-53.97,-78.19,5.28,4.56,2.95,2.61,2.26,5.06,4.33,46.19,36.00,98.41,95.38,0,802.33,68.22,43755.12,1803,1201.97,2309.41,53.50,46.50,86.91,29.87,55.83,48.17,3.51,0.49,0.96,8.75,1.02,176083.10,2.46,5.19,0.00,0.00,2011,6,21,96400,86900,92581,92200,13038907,12070557806
4,삼성전기(주),9150,2011/12,7.21,0.93,-13.45,-58.76,-36.99,-88.49,5.32,6.51,4.68,4.53,5.54,8.36,10.01,82.69,90.07,98.38,94.27,0,805.90,73.41,77729.24,4510,6302.39,4134.72,52.88,47.12,89.09,21.70,41.03,47.69,9.92,0.85,1.57,15.55,1.86,1637513.33,22.34,27.15,49.04,50.96,2011,12,21,91200,77000,84738,77700,18663868,15827876707


In [18]:
ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_ssem.columns = ['지표','종가']
df_ssem

,지표,종가
47,종가,1.000000
44,최고가,0.992346
46,산술평균,0.990793
45,최저가,0.984799
49,거래대금,0.915302
7,[공통]매출액정상영업이익률(IFRS연결),0.851472
14,[공통]자본금정상영업이익률(IFRS연결),0.736894
12,[공통]자기자본정상영업이익률(IFRS연결),0.722063
10,[공통]총자본정상영업이익률(IFRS연결),0.709207
9,[공통]총자본사업이익률(IFRS연결),0.705414


In [19]:
df_ssem.index

Int64Index([47, 44, 46, 45, 49, 7, 14, 12, 10, 9, 26, 8, 27, 48, 24, 13, 11, 3, 15, 29, 22, 5, 33, 2, 28, 23, 32, 1, 6, 34, 35, 31, 40, 21, 42, 4, 43, 37, 36, 41, 20, 38, 19, 39, 30, 17, 25, 16, 0, 18], dtype='int64')

In [16]:
df_ssem.to_excel('ssem.xlsx')

# 6개월 후 주가

In [44]:
# 3개월 후 주가
# ssg_1['종가(원)'] = ssg_1['종가(원)'][1:].reset_index(drop=True)

In [45]:
ssg_1['종가(원)'] = ssg_1['종가(원)'][2:].reset_index(drop=True)

In [46]:
ssg_1['종가(원)']

0     537000.0
1     614000.0
2     339500.0
3     245000.0
4     210000.0
5     218500.0
6     211000.0
7     252500.0
8     218500.0
9     181000.0
10    268000.0
11    230000.0
12    195000.0
13    176000.0
14    234500.0
15    300000.0
16    401500.0
17         NaN
18         NaN
Name: 종가(원), dtype: float64

In [47]:
ssg_1 = ssg_1.dropna(axis = 0)
ssg_1

,총자본증가율(IFRS연결),자기자본증가율(IFRS연결),매출액증가율(IFRS연결),정상영업이익증가율(IFRS연결),순이익증가율(IFRS연결),총포괄이익증가율(IFRS연결),매출액정상영업이익률(IFRS연결),매출액순이익률(IFRS연결),총자본사업이익률(IFRS연결),총자본정상영업이익률(IFRS연결),총자본순이익률(IFRS연결),자기자본정상영업이익률(IFRS연결),자기자본순이익률(IFRS연결),자본금정상영업이익률(IFRS연결),자본금순이익률(IFRS연결),영업비용 대 영업수익비율(IFRS연결),수지비율(관계기업투자손익 제외)(IFRS연결),세금과공과 대 총비용비율(IFRS연결),유보율(IFRS연결),적립금비율(재정비율)(IFRS연결),1주당매출액(IFRS연결)(원),1주당순이익(IFRS연결)(원),1주당 CASH FLOW(IFRS연결)(원),1주당정상영업이익(IFRS연결)(원),자기자본구성비율(IFRS연결),타인자본구성비율(IFRS연결),부채비율(IFRS연결),차입금의존도(IFRS연결),차입금비율(IFRS연결),유보액대비율(IFRS연결),CASH FLOW 대 총자본비율(IFRS연결),총자본회전률(IFRS연결),자기자본회전률(IFRS연결),자본금회전률(IFRS연결),타인자본회전률(IFRS연결),부가가치(IFRS연결)(백만원),총자본투자효율(IFRS연결),부가가치율(IFRS연결),노동소득분배율(IFRS연결),자본분배율(IFRS연결),거래년도(*),거래월(*),거래일수,최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
0,0.00,0.00,0.00,0.00,0.00,0.00,7.93,5.02,9.43,9.22,5.83,23.84,15.07,873.59,530.86,92.07,93.14,0,3421.60,97.28,550477.53,26543,51310.85,43679.67,38.68,61.32,158.52,34.46,89.07,37.63,10.83,1.16,3.00,110.10,1.90,849625.26,9.51,8.18,0.00,0.00,2007,12,18,763000,666000,719444,537000.0,1074834,7725748911
1,20.85,16.02,6.45,9.98,14.07,0.00,8.20,5.09,9.43,9.18,5.71,24.40,15.92,960.76,605.55,91.80,93.60,0,3985.88,98.07,585991.43,30278,31790.50,48037.80,36.77,63.23,171.93,41.06,111.66,35.90,7.26,1.12,2.98,117.20,1.80,909906.91,8.43,8.23,0.00,0.00,2008,12,21,498500,386000,451476,614000.0,1571615,7070606603
2,3.31,15.15,4.00,2.99,-0.82,0.00,8.12,5.07,8.78,8.50,5.31,21.86,13.66,989.53,600.61,91.88,93.32,0,4605.05,97.34,609403.28,30030,51038.02,49476.31,40.96,59.04,144.15,37.22,90.87,40.11,7.00,1.05,2.69,121.88,1.71,965271.72,8.65,8.40,0.00,0.00,2009,12,21,566000,515000,537571,339500.0,1292884,6968542198
3,28.46,64.19,-83.75,-74.39,92.00,0.00,12.80,59.48,2.09,1.88,8.72,3.98,18.55,253.46,1153.16,87.83,81.19,0,7625.18,74.26,99043.79,0,25740.79,12673.04,51.87,48.13,92.78,22.98,44.31,51.21,5.05,0.15,0.31,19.81,0.28,763580.34,5.33,40.88,22.39,77.61,2010,12,22,672000,554000,578682,245000.0,987961,5775921269
4,0.00,0.00,0.00,0.00,0.00,0.00,12.84,312.14,8.25,7.18,174.47,13.62,344.69,574.06,13913.80,88.04,86.46,0,3943.89,423.30,111732.27,120246,-31268.35,14351.61,52.72,47.28,89.66,20.49,38.87,51.56,-3.91,0.56,1.06,44.69,1.18,346313.21,8.80,31.48,25.94,74.06,2011,6,21,407500,270000,318095,210000.0,1503624,5239209057
5,-71.07,-73.03,16.99,11.96,221.99,17.91,12.24,161.16,3.20,2.90,38.13,5.57,75.71,372.88,4879.25,88.89,87.87,0,3898.07,431.77,221984.54,0,-6074.14,27180.56,52.59,47.41,90.15,19.37,36.83,51.49,5.14,0.24,0.45,30.45,0.49,695774.23,16.79,31.84,27.38,72.62,2011,12,21,262000,234000,246024,218500.0,816130,2002264137
6,12.45,6.55,-0.32,-7.37,-97.17,-95.20,11.94,9.84,3.68,3.13,2.58,5.92,4.72,265.88,196.92,89.69,87.27,0,4208.37,72.51,111372.82,9855,-1832.37,13293.99,52.93,47.07,88.92,22.44,42.40,51.90,-3.68,0.26,0.50,22.27,0.56,366473.56,8.28,33.42,27.22,72.78,2012,6,20,235500,205000,218350,211000.0,713648,1559516030
7,68.13,12.35,5.09,-11.62,-95.38,-93.24,10.30,8.44,4.81,4.26,3.49,8.96,7.75,480.43,328.29,89.70,88.74,0,4390.99,73.08,233281.94,16429,25823.04,24021.46,44.50,55.50,124.74,30.51,68.57,43.84,1.40,0.41,0.87,46.66,0.79,764865.55,10.98,33.30,27.42,72.58,2012,12,18,225500,201500,215167,252500.0,439529,944658850
8,66.89,11.53,7.49,13.54,-10.57,1.45,12.61,8.51,2.85,2.52,1.70,5.31,3.87,301.88,176.10,87.39,89.45,0,4704.19,71.57,119711.06,8813,-372.55,15094.19,44.11,55.89,126.69,33.63,76.24,43.50,-0.15,0.20,0.42,23.94,0.38,416986.58,5.64,35.38,27.23,72.77,2013,6,19,235500,206500,218526,218500.0,811477,1771449119
9,7.63,9.20,6.31,29.55,-2.47,-6.35,12.55,7.97,4.61,4.24,2.69,9.51,6.83,622.42,320.19,87.45,89.42,0,4803.58,73.09,248003.25,16024,27825.83,31120.80,44.58,55.42,124.29,31.89,71.52,43.98,3.52,0.34,0.76,49.60,0.61,865821.60,11.54,35.46,27.26,72.74,2013,12,20,258500,239000,248300,181000.0,471697,1167677110


In [ ]:
ssg_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(ssg_corr['종가'].sort_values(ascending = False))
df_nc

In [22]:
# ssg_pct = ssg_1.pct_change()

In [23]:
# ssg_pct_corr = ssg_pct.corr(method = 'pearson')

In [26]:
# ssg_pct_corr['종가(원)'].sort_values(ascending = False)

종가(원)                        1.000000
최고가(원)                       0.970108
산술평균(원)                      0.961772
최저가(원)                       0.910999
거래대금(원)                      0.625513
자본분배율(IFRS연결)                0.500625
거래량(주)                       0.433844
총포괄이익증가율(IFRS연결)             0.382048
자기자본증가율(IFRS연결)              0.372977
유보율(IFRS연결)                  0.363243
유보액대비율(IFRS연결)               0.276536
자기자본구성비율(IFRS연결)             0.267535
1주당순이익(IFRS연결)(원)            0.265287
부가가치율(IFRS연결)                0.241428
총자본증가율(IFRS연결)               0.219269
매출액증가율(IFRS연결)               0.115088
매출액순이익률(IFRS연결)              0.090200
거래년도(*)                      0.075765
매출액정상영업이익률(IFRS연결)           0.073447
부가가치(IFRS연결)(백만원)            0.007244
CASH FLOW 대 총자본비율(IFRS연결)   -0.005939
1주당 CASH FLOW(IFRS연결)(원)    -0.078514
영업비용 대 영업수익비율(IFRS연결)       -0.082139
거래월(*)                      -0.094322
차입금의존도(IFRS연결)              -0.095866
거래일수                        -0.103199
정상영업이익증가율(IF

In [7]:
ssg_corr = ssg.corr(method = 'pearson')

In [9]:
df_ssg = pd.DataFrame(ssg_corr['종가(원)'].sort_values(ascending = False))

In [10]:
df_ssg.to_excel('ssg.xlsx')